# RSA

Rivest Shamir Adleman (RSA) jest jednym z pierwszych algorytmów asymetrycznych. Został zaprojektowany w 1977 roku i jest używany do dzisiejszego dnia. Zyskał akceptację zarówno NISTu jak i organizacji ISO/IEC oraz RFC. RSA posiada parę kluczy - publiczny oraz prywatny. Publiczny klucz może być znany każdemu i służy on do operacji szyfrowania. Klucz prywatny jest znany tylko i wyłącznie instancji, która klucze generowała. Ta sama instancja jako jedna jedyna ma możliwość odszyfrowania kryptogramów.

RSA umożliwia także tworzenie podpisów cyfrowych (z ang *Digital Signatures*, czyli *DS*). Podpis cyfrowy to dodatkowy blok informacji dołączony do wiadomości, który zapewnia:
1. *Integrity* - integralność wiadomości, czyli potwierdzenie, że nie była ona w żaden sposób modyfikowana.
2. *Authentication* - autentykacje podpisującego, czyli potwierdzenie jego tożsamości.
3. *Non-repudiation* - czyli wysyłający podpisaną wiadomość nie ma możliwości zaprzeczenia faktu, że to on ją podpisał, natomiast otrzymujący wiadomość nie ma możliwości zaprzeczenia faktu, iż to on ją zweryfikował. 

Samo haszowanie wiadomości zapewnia tylko *integirty*, natomiast utworzenie kodu MAC (*Message Authentiaction Code*) zapewnia jedynie *integrity* oraz *authentiaction*.

Tworzenie podpisu cyfrowego z wykorzystaniem RSA wygląda odwrotnie niż komunikacja szyfrowana. To znaczy: podpis tworzony jest z wykorzystaniem klucza prywatnego - a więc tylko instancja generująca klucze może wiadomość podpisać. Weryfikacja odbywa się z wykorzystaniem klucza publicznego - czyli każda instancja, której nasz klucz udostępnimy, może podpis zweryfikować. 

Na początek zaimportujmy niezbędne biblioteki.

In [209]:
# !pip install pycryptodome
import Crypto.Util.number as cu
import hashlib as hl
import math
import numpy as np

**Zadanie 1**

Odpowiedz na poniższe pytania. Rozważ sytuację, w której dwie instancje komunikują się ze sobą, a trzecia próbuje dokonywać nieautoryzowanych zmian w treści wiadomości na drodze jej przesyłu.
1. Dlaczego haszowanie zapewnia tylko integrity? Podpowiedź: czy haszowanie uwzględnia jakikolwiek klucz prywatny?
2. Dlaczego kod MAC nie zapewnia *non-repudiation*? Co (niepożądanego) może dokonać odbierający wiadomość i atakujący komunikację? Podpowiedź: czy kody MAC, w szczególności popularny kod HMAC - który zakłada użycie klucza prywatnego - w jakikolwiek sposób weryfikuje KTO i DO CZEGO go użył? Kto dysponuje, a kto NIE dysponuje kluczem prywatnym HMAC?
3. Dlaczego podpis cyfrowy zapewnia wszystkie te trzy cechy?

## Generowanie kluczy

Algorytm generowania kluczy RSA może zostać przedstawiony w następujący sposób:

1) Znajdź dwie różne i kryptograficznie bezpieczne liczby pierwsze.

2) Oblicz $n = p * q$.

3) Oblicz $f = (p - 1) * (q - 1)$.

4) Znajdź dowolne $e$, takie, że $1 < e < f$ oraz $GCD(f, e) = 1$. GCD to największy wspólny dzielnik. Para $(e, n)$ to jest **klucz publiczny**.

5) Oblicz $d = e^{-1}$ mod $f$. Para $(d, n)$ to **klucz prywatny**, przy czym tajne jest tylko $d$. 

W ten sposób generowane parametry byłyby matematycznie poprawne, lecz kryptograficznie niebezpieczne. Ustalmy więc, że chcemy aby nasz klucz publiczny był odpwowiednio długi. Będzie to długość bitowa parametru $n$, oznaczmy ją jako $nlen = 2048$. Parametr $nlen$ zawsze przyjmuje parzyste wartości. Mając to założenie, musimy (**uwzględniając wszystkie założenia z algorytmu generowania kluczy**) dodatkowo zapewnić, że:

1. $65537 ≤ e < 2^{256}$
2. $LCM(p - 1, q - 1) \geq e * 2^{nlen/2}$
3. $2^{(nlen - 1)//2} < p < 2^{nlen/2}$
4. $2^{(nlen - 1)//2} < q < 2^{nlen/2}$
5. $|p - q| > 2^{(nlen/2) - 100}$

Gdzie LCM oznacza *Least Common Multiple*, czyli najmniejszą wspólną wielokrotność. Funkcję LCM znajdziesz w bibliotece math. Do potęgowania **nie używaj** pythonowej notacji "**", tylko metody pow() - przetestuj obie te metody obliczania potęgi i porównaj wydajność (zadanie opcjonalne). Do obliczania wartości bezwzględnej użyj metody abs() - również standardowa metoda pythona. Resztę niezbędnych metod znajdziesz w bibliotece [Crypto.Util.number](https://pycryptodome.readthedocs.io/en/latest/src/util/util.html) zaimportowanej jako cu. Opis powyższych założeń możesz znaleźć w [tym](https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-56Br2.pdf) dokumencie NIST-owym.

**Zadanie 2**

Uwzględniając wszystko powyższe, napisz metodę/metody generujące wartości $n$, $e$ oraz $d$.

In [210]:
# your code here
def generate_rsa_pair(nlen):
    assert nlen % 2 == 0

    # 1, 2
    p, q = 1, 1
    n = p * q
    while not p != q or\
          not len(bin(n)) - 2 != nlen or\
          not (nlen - 1)//2 < math.log2(p) < nlen/2 or\
              not (nlen - 1)//2 < math.log2(q) < nlen/2 or\
                not math.log2(abs(p - q)) > nlen/2 - 100 :
        p = cu.getPrime(nlen // 2)
        q = cu.getPrime(nlen // 2)
        n = p * q

    # 3
    f = (p - 1) * (q - 1)
    # 4
    # e = f - 1
    e = 65537
    while math.gcd(e, f) != 1:
        e += 2
    
    d = cu.inverse(e, f)

    return (d, n), (e, n)
    

In [211]:
generate_rsa_pair(2048)

((12479741933901292070156215969787812924097116551269241801086129352065973755764192118629780256663467792869177508177703592080339959075040390595055440953846233168247978728583342148806656449520760364120334317740816430934918842379491858677287501180539577400782288091962131313470651650440388887031378661211868911405920626999227018212313514813923947358542629128195382857831970508578183779721056538599280458472211231468223504279784715136949753851478072739258923788151880930182993239571714986791672640316693596242475179991140515975805720357826689602530897744090847002304556474645610622483507790396100849389114018926378943651833,
  14550263242463022867442812367899234947013088673400798774577603304448377044199851610514666322978664118077730094704633618227201791426961307901089616512643798956564867764942208720825849811108894557708533030577287122345835800342010601877450871980022100373860433272543135683245736550050911147095327663977553376506794854027467287896879607150526768894326093111358802893940989742552593

## Naiwne szyfrowanie i deszyfrowanie


Naiwny algorytm szyfrowania wiadomości **M** z wykorzystaniem RSA:

1) Zakoduj $M$ jako liczbę.

2) Oblicz: $C = M^e$ mod $n$.

Naiwny algorytm deszyfrowania kryptogramu **C** z wykorzystaniem RSA:

1) $M = C^d$ mod $n$.

2) Zdekoduj wiadomość $M$ do jej pierwotnej postaci (np. stringa). 


**Zadanie 3**

Napisz metody szyfrujące i deszyfrujące wiadomość $M$ zgodnie z powyższym algorytmem. Zaszyfruj wiadomość, zdeszyfruj i wypisz oryginalny tekst na ekranie. Odpowiedz na pytanie: jaki warunek musi spełniać liczbowa reprezentacja wiadomości $M$, aby można ją było poprawnie zaszyfrować i zdeszyfrować?

In [212]:
priv_k, pub_k = generate_rsa_pair(2048)

In [213]:
def encrypt_with_rsa(M, public):
    m = cu.bytes_to_long(M)
    mlen = len(M)
    e, n = public
    C = pow(m, e, n)
    return cu.long_to_bytes(C, mlen)


def decrypt_with_rsa(C, private):
    clen = len(C)
    C = cu.bytes_to_long(C)
    d, n = private
    M = pow(C, d, n)
    return cu.long_to_bytes(M, clen)



In [214]:
M = "Ala ma kota"

print(f"message: {M}")
C = encrypt_with_rsa(M.encode(), pub_k)
print(f"cipher   : {C}")
D = decrypt_with_rsa(C, priv_k)
print(f"decrypted: {D}")

message: Ala ma kota
cipher   : b'\x00\x00\x00\x00\x00\x00\x00\x005\xfb\xb8\x9f)\xb2b\r=Z\x82\x12\x06`\x03\xeb0\x08u&\x9a\x85\x8f\xc9K\xfc\xb90\xc5v385XL\xaer\xba\xf6e\x14\xca\xb3f\xc8\xed\xfd!\x920\xbf3"\xbd:x:|\xd7\xa2\x96\x0f\xfb\x9d\xce\x1eCAo\xa90\x95\x97.\xb3\xe1\x0b\x11\x83\xd3\xc9\x15\xc7+\x8b\xebh\xec\xda\x8d\xec[[T\x13\x081\xb0\x13|\xaa\xa8A\xea\xa4\x1cv\xf5NF\x9b8\xd94\xe69=\xee\xc4gK&\xbc\x06\x92\xf4\x0e\x963U\xbe\xac\xf0\xb3\xff\x0e\xc1\x7f\x84\xc0\xb7\t\xdc\xb5h\xdbE`\xc3\xabzO\xbf\x03\xd2\x1dlt\xd41\xeb$.\x8e\xd2\xf7\x0e\x97\xaab\xf9O%\x02\xab\xfc\xfb\x9b\'\xbd\xfe<\x99(\xb3\xec\xcfC\xd1\x82A_-"\xfa\x12L\x089w\xa8\xcfb)\x11\x1aSN\xf0\x9c\xa4\xd7/\xf9\x85\xb6;\xe8Q \xa5\xfbzb|u\x945\xa6\x0f\x11\xe0\x85\x8e9-\xfe\xd8M\xea\x0b]>,\xe0\x81\xa4\xa0&\xb2\xa3\xc7\xf6\xd1\xf8\x0e'
decrypted: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

## Naiwny schemat podpisu cyfrowego

Naiwna metoda tworzenia podpisu z wiadmości $M$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $SIG = h^d$ mod $n$.

4) Wyślij parę $(M, SIG)$ weryfikującemu.

Naiwna metoda weryfikacji podpisu $(M, SIG)$:

1) Oblicz $h = H(M)$. H to uzgodniona funkcja skrótu, niech to będzie SHA-256.

2) Zakoduj $h$ jako liczbę.

3) Oblicz $VER = SIG^e$ mod $n$.

4) Jeżeli $VER = h$, weryfikacja przebiegła pomyślnie, a w przeciwnym razie niepomyślnie.

**Zadanie 4**

Zaimplementuj naiwną metodę tworzenia i weryfikowania podpisu cyfrowego RSA.

In [215]:
# Your code here
def naive_fingerprint(M, priv_k):
    d, n = priv_k
    h = hl.sha3_256(M.encode()).hexdigest()
    h = int(h, 16)
    sig = pow(h, d, n)
    return M, sig
def naive_verify_fingerprint(M, sig, pub_k):
    e, n = pub_k
    h = hl.sha3_256(M.encode()).hexdigest()
    h = int(h, 16)
    ver = pow(sig, e, n)
    return ver == h

In [216]:
M, sig = naive_fingerprint(M, priv_k)
naive_verify_fingerprint(M, sig, pub_k)

True

## MGF 1

W dalszej części laboratoriów będziemy potrzebowali generować maskę. Jedynym zatwierdzonym algorytmem który do tego służy jest *Mask Generation Function 1*, opisany w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017). Jest on stosunkowo prosty. 

Parametry wejściowe:

1) M - bajty wiadomości.

2) len - pożądana długość zwórconej maski w bajtach.

3) H - wybrana funkcja skrótu, zwracająca $n$ bitowy skrót. Niech to będzie SHA-256. Dla wygody przyjmijmy też, że $hlen = n / 8$ oznacza liczbę bajtów zwracaną przez naszą funkcję skrótu.

Wyjściem funkcji są bajty tworzące maskę.

Algorytm MGF-1:

1) Dla 32-biotwego integera $i = 0, ..., ⌈ \frac{len}{hlen}⌉ - 1$ wykonuj kroki 2 i 3.

2) Oblicz tmp = H(M || i). Znak || to konkatenacja i chodzi tu o bajty wiadomości M oraz reprezentację w bajtach 32-bitowego itegera $i$.

3) Oblicz output = output || tmp.

4) Zwróc $len$ wiodących bajtów zmiennej output.

**Zadanie 5**

Zaprogramuj i przetestuj dla dowolnych wartości funkcję MGF1.

In [217]:
# Your code here
def mgf1(M, rlen=256, H=None, hlen=0):
    if H is None:
        H = lambda b : hl.sha3_256(b).hexdigest()
        hlen = 256 / 8 # TODO
    if type(M) == str:
        Mb = M.encode()
    else:
        Mb = M

    out = bytes()
    for i in range(math.ceil(rlen / hlen)):
        tmp = H(Mb + cu.long_to_bytes(1, 4))
        tmp = cu.long_to_bytes(int(tmp, 16))
        out += tmp
    return out[:rlen]

In [218]:
mgf1(M.encode())

b'#$1\x0e\xfe"\xa3\xe35\xc0\x11\x00S\x93\xbcr\x05\x9dE\xe4\xb7}F\x1e\x98\xf8>s\xa9\xa4V\x8d#$1\x0e\xfe"\xa3\xe35\xc0\x11\x00S\x93\xbcr\x05\x9dE\xe4\xb7}F\x1e\x98\xf8>s\xa9\xa4V\x8d#$1\x0e\xfe"\xa3\xe35\xc0\x11\x00S\x93\xbcr\x05\x9dE\xe4\xb7}F\x1e\x98\xf8>s\xa9\xa4V\x8d#$1\x0e\xfe"\xa3\xe35\xc0\x11\x00S\x93\xbcr\x05\x9dE\xe4\xb7}F\x1e\x98\xf8>s\xa9\xa4V\x8d#$1\x0e\xfe"\xa3\xe35\xc0\x11\x00S\x93\xbcr\x05\x9dE\xe4\xb7}F\x1e\x98\xf8>s\xa9\xa4V\x8d#$1\x0e\xfe"\xa3\xe35\xc0\x11\x00S\x93\xbcr\x05\x9dE\xe4\xb7}F\x1e\x98\xf8>s\xa9\xa4V\x8d#$1\x0e\xfe"\xa3\xe35\xc0\x11\x00S\x93\xbcr\x05\x9dE\xe4\xb7}F\x1e\x98\xf8>s\xa9\xa4V\x8d#$1\x0e\xfe"\xa3\xe35\xc0\x11\x00S\x93\xbcr\x05\x9dE\xe4\xb7}F\x1e\x98\xf8>s\xa9\xa4V\x8d'

## OAEP

Nasz schemat ma na ten moment jedną sporą wadę, mianowicie rozmiar szyfrowanej wiadomości może być zbyt mały, czyniąc algorytm mniej bezpiecznym. Aby tego uniknąć, używamy algorytmu paddingu opisanego w [RFC 8017](https://www.rfc-editor.org/rfc/rfc8017#section-8), który zwie się *Optimal Assymetric Encryption Padding*.

### OAEP encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - długość liczby $n$ wyrażona w bajtach.

3) $mlen$ - długość wiadomości wyrażona w bajtach.

4) $M$ - bajty wiadomości.

5) $mgf1$ - Mask Generation Function 1.

Algorytm:

1) Jeżeli $mlen > k - 2*hlen - 2$ zwróc błąd.

2) Oblicz: $lHash = H("")$.

3) Wygeneruj tablicę bajtów $PS$ składającą się z $k - mlen - 2*hlen - 2$ bajtów o wartości 0x00. Rozmiar $PS$ może wynosić 0.

4) Oblicz: $DB = lHash || PS || 0x01 || M$. Długość $DB$ powinna wynosić $k - hlen - 1$ bajtów.

5) Wygeneruj losową tablicę bajtów $seed$ o rozmiarze $hlen$.

6) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

7) Oblicz: $maskedDB = DB ⊕ dbMask$.

8) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

9) Oblicz: $maskedSeed = seed ⊕ seedMask$.

10) Oblicz: $EM = 0x00 || maskedSeed || maskedDB$. Długość $EM$ powinna wynosić $k$.

11) Zwróc $EM$.

### OAEP decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $k$ - rozmiar EM wyrażony w bajtach.

3) $mgf1$ - Mask Generation Function 1.

4) $EM$ - bajty zakodowanej wiadomości.

Algorytm:

1) Rozpakuj tablicę bajtów $EM$. Jej pierwszy bajt (najbardziej znaczący) przypisz do $Y$. Kolejne $hlen$ bajtów przypisz do $maskedSeed$, resztę do $maskedDB$. Czyli $EM = Y || maskedSeed || maskedDB$.

2) Oblicz: $lHash = H("")$.

3) Oblicz: $seedMask = mgf1(maskedDB, hlen)$.

4) Oblicz: $seed = maskedSeed ⊕ seedMask$.

5) Oblicz: $dbMask = mgf1(seed, k - hlen - 1)$.

6) Oblicz: $DB = maskedDB ⊕ dbMask$.

7) Rozpkauj tablicę bakjtów $DB$. Pierwsze (najbardziej znaczące) $hlen$ bajtów przypisz do zmiennej $lHash'$. Następne $k - mlen - 2*hlen - 2$ bajtów do PS. Kolejny pojedynczy bajt powinien wynosić 0x01, jeżeli jest inaczej zwróć błąd i **zakończ działanie**. Resztę bajtów przypsiz do zmiennej $M$. Czyli: $DB = lHash' || PS || 0x01 || M$. 

8) Jeżeli $Y \neq 0x00$ zwróć błąd i **zakończ działanie**.

9) Jeżeli $lHash \neq lHash'$ zwróć błąd i **zakończ działanie**.

10) Zwróc $M$.

**Zadanie 6**

Zaproogramuj kodowanie i dekodowanie OAEP. Zmodyfikuj algorytm szyfrowania RSA, tak, aby przed zaszyfrowaniem wiadomość była paddingowana. Zmodyfikuj algorytm deszyfrowania tak, aby po zdeszyfrowaniu konieczne było wywołanie metody dekodowania OAEP w celu odzyskania wiadomości.

In [219]:
M = "Ala ma kota"

# Your code here
def enc_with_oaep(M, pub_k, k=256, H=None, hlen=None):
    if H is None:
        H = lambda mb: hl.sha3_256(mb).digest()
        hlen = 256 // 8

    if type(M) == str:
        Mb = M.encode()
    else:
        Mb = M
        
    mlen = len(Mb)
    assert not mlen > k - 2 * hlen - 2

    lHash = H("".encode())
    PS = bytes([0x00] * (k - mlen - 2*hlen - 2))
    DB = lHash + PS + bytes([0x01]) + Mb
    assert len(DB) == k - hlen - 1

    seed = [cu.getRandomInteger(8) for _ in range(hlen)]
    seed = bytes(seed)
    dbMask = mgf1(seed, k - hlen - 1)
    maskedDB = bytes(np.uint8(list(DB)) ^ np.uint8(list(dbMask)))
    seedMask = mgf1(maskedDB, hlen)
    maskedSeed = bytes(np.uint8(list(seed)) ^ np.uint8(list(seedMask)))
    EM = bytes([0x00]) + maskedSeed + maskedDB
    assert len(EM) == k

    return encrypt_with_rsa(EM, pub_k)
    # return cu.long_to_bytes(pow(cu.bytes_to_long(EM), *pub_k))


def dec_with_oaep(EM: bytes, priv_k, mlen, k=256, H=None, hlen=None):
    if H is None:
        H = lambda mb: hl.sha3_256(mb).digest()
        hlen = 256 // 8
    
    EM = decrypt_with_rsa(EM, priv_k)

    Y = EM[0]
    maskedSeed = EM[1:hlen+1]
    maskedDB = EM[hlen + 1:]
    lHash = H("".encode())
    seedMask = mgf1(maskedDB, hlen)
    seed = bytes([a ^ b for a, b in zip(maskedSeed, seedMask)])
    dbMask = mgf1(seed, k - hlen - 1)
    DB = bytes([a ^ b for a,b in zip(maskedDB, dbMask)])

    lHash_ = DB[:hlen]
    offset = hlen
    nex = k - mlen - 2*hlen - 2
    PS = DB[offset: offset + nex]
    offset += nex
    assert DB[offset] == 0x01
    offset += 1
    M = DB[offset:]

    assert Y == 0x00
    assert lHash == lHash_

    return M


In [220]:
EM = enc_with_oaep(M, pub_k)
D = dec_with_oaep(EM, priv_k, mlen=len(M))
print(f"Message: {M}")
print(f"Aft. OAEP: {EM}")
print(f"Ret. OAEP: {D}")

Message: Ala ma kota
Aft. OAEP: b'?\xbe\x81\xac\xab\xb8\x90\x98\x17@\x03\xfd]\x972\xce)\xfe\x00p\xb2-\x1et\xfa6\x83\t\xe5\xcco\xfe"#\n\xf0\x0f\x19B\xe2_\x80\x81$\xa3\x88\xf1\x1b\x07\x9cy\xd6\xbc\x8d;\xe7\x165\xbe\xbe\xb9\xcb\x91\x9fL\xb9=\xda\xf4\x88\r\xa4&\x89\x9f\x9f\x95s\xa3\xe4\xd0\x95\xe9)\x08\xaa\x833\x85\xd88\x8a\x9d\xd5\xe8\xe5\x118@^\xf1\n\xe1\xfe\x8f\xb5%R\x04`_\xd1\x9aNne\x1f\x1e\x8f>H\xb0\xd5\xdb\xe9um\xdf\xd3zH(\xaf\xeb\xb0^\x16K\xe4\x08=+_\xec\xff\xe0\x10\x03)\xc9\xce\\L==\x87N\xfb6\x80\n\xdb*\xfb\xa5\xa2\xac\x19\x92\xeb\xdb\x9f\xd0&\xc15\xc8\x91@\xa1\xd9d\x84\xca\x9fY\xdf\x17b\xcd\xb5T\xb0&\xff\xdf\xa1\xc6\xe8\x8d\xdb\xea/f\xc7\xbc\xaei\r\xc8B\xac\x9fe\xb7\'\x93\xbeE`\x90\x1a\xcf\xda\x1a\xce\r\xd3{\x15\xaeRba\xe7\xdd^c\xa8\xcd\x0f|\xe1 A\xb2xXl`\xcc\xfb\xbf\xa7<\x8a'
Ret. OAEP: b'Ala ma kota'


## EMSA - PSS

Utworzenie bezpiecznej sygnatury RSA wymaga zastowania algorytmu *Encoding Method for Signature with Appendix - Probabilistic Signature Scheme* .

### EMSA encoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $M$ - bajty wiadomości do podpisania.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - pożądana długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.


Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd**.

3) Wygeneruj tablicę losowych bajtów $salt$ o długości $slen$.

4) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

5) Oblicz: $mHash' = H(M')$.

6) Wygeneruj tablicę $PS$ składającą się z bajtów 0x00 o długości $emlen - slen - hlen - 2$.

7) Oblicz: $DB = PS || 0x01 || salt$. Długość $DB$ powinna wynosić $emlen - hlen - 1$ bajtów.

8) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

9) Oblicz: $maskedDB = DB ⊕ dbMask$.

10) Ustaw $8 * emlen - emBits$ **najbardziej znaczących** bitów $maskedDB$ na wartości 0.

11) Oblicz: $EM = maskedDB || mHash' || 0xbc$.

12) Zwróć $EM$.


## EMSA decoding

Parametry wejściowe:

1) $H$ - funkcja skrótu SHA-256, oraz $hlen$ czyli długość zwracanego skrótu w bajtach.

2) $slen$ - długość soli w bajtach, powinna być równa $hlen$.

3) $EM$ - sygnatura wiadomości $M$.

4) $M$ - bajty wiadomości do weryfikacji.

4) $mgf1$ - Mask Generation Function 1.

5) $emBits$ - długość sygnatury w bitach. Jest to najczęściej długość bitowa liczby modulus $n$ pomniejszona o jeden, czyli w naszym przypadku 2047.

6) $emlen$ - długość sygnatury w bajtach, równa długości parametru $n$ wyrażonego w bajtach.

Algorytm:

1) Oblicz: $mHash = H(M)$.

2) Jeżeli $emlen < hlen + slen + 2$ **zakończ i zwróć błąd weryfikacji**.

3) Jeżeli ostatni bajt (najmniej znaczący) $EM$ nie ma wartości 0xbc **zakończ i zwróć błąd weryfikacji**.

4) Podstaw $emlen - hlen - 1$ najbardziej znaczących bajtów do $maskedDB$ oraz kolejne $hlen$ bajtów do $mHash'$.

5) Jeżeli $8 * emlen - emBits$ najbardziej znaczących bitów $maskedDB$ nie ma wartości 0, **zakończ i zwróć błąd weryfikacji**.

6) Oblicz: $dbMask = mgf1(mHash', emlen - hlen - 1)$.

7) Oblicz: $DB = maskedDB ⊕ dbMask$.

8) Ustaw $8 * emlen - emBits$ najbardziej znaczących bitów $DB$ na 0.

9) Jeżeli $emlen - hlen - slen - 2$ najbardziej znaczących bajtów $DB$ nie posiada wartości 0x00 lub gdy bajt na pozycji $emlen - hlen - slen - 1$ (licząc od najbardziej znaczącego) nie posiada wartości 0x01 **zakończ i zwróć błąd weryfikacji**.

10) Przypisz do zmiennej $salt$ dokładnie $slen$ najmniej znaczących bajtów $DB$.

11) Oblicz: $M' = 0x00 00 00 00 00 00 00 00 || mHash || salt$. Długość $M'$ to $8 + hlen + slen$.

12) Oblicz $mHash'' = H(M')$.

13) Jeżeli $mHash' \neq mHash''$ **zakończ i zwróć błąd weryfikacji**, w przeciwnym razie **weryfikacja powiodła się**.


**Zadanie 7**

Zaprogramuj kodowanie i dekodowanie EMSA, a następnie popraw algorytmy tworzenia i weryfikacji podpisu cyfrowego RSA. Tworzenie podpisu powinno wyglądać tak, że wiadomość najpierw jest kodowana z wykorzystaniem EMSA, a później tworzony jest popdis z wykorzystaniem klucza prywatnego. Dekodowanie powinno wyglądać tak, że najpierw używany jest klucz publiczny do odtworzenia podpisu EMSA, a następnie wykorzystywane jest dekodowanie EMSA w celu weryfikacji.

In [221]:
# Your code here
def enc_emsa_pss(M, slen, emBits, emlen, H=None):
    if H is None:
        H = lambda mb: hl.sha3_256(mb).digest()
        hlen = 256 // 8
    
    if type(M) == str:
        Mb = M.encode()
    else:
        Mb = M
        
    mHash = H(Mb)
    assert emlen >= hlen + slen + 2
    salt = cu.Random.get_random_bytes(slen)

    Mb_ = bytes([0x00] * 8) + mHash + salt
    assert len(Mb_) == 8 + hlen + slen
    mHash_ = H(Mb_)
    PS = bytes([0x00]* (emlen - slen - hlen - 2))
    DB = PS + bytes([0x01]) + salt
    assert len(DB) == emlen - hlen - 1
    assert DB[emlen - hlen - slen - 2] == 1
    
    dbMask = mgf1(mHash_, emlen - hlen - 1)
    maskedDB = np.array([a ^ b for a,b in zip(DB, dbMask)], dtype=np.uint8)
    maskedDB[:8 * emlen - emBits] = 0
    maskedDB = bytes(maskedDB)
    EM = maskedDB + mHash_ + bytes([0xbc])
    return EM

def dec_emsa_pss(EM, M, slen, emBits, emlen, H=None):
    if H is None:
        H = lambda mb: hl.sha3_256(mb).digest()
        hlen = 256 // 8
    
    if type(M) == str:
        Mb = M.encode()
    else:
        Mb = M
        
    mHash = H(Mb)
    assert emlen >= hlen + slen + 2
    assert EM[-1] == 0xbc
    s = emlen - hlen - 1
    maskedDB, mHash_ = EM[:s], EM[s:s+hlen]
    assert sum(list(maskedDB[:8*emlen - emBits])) == 0
    dbMask = mgf1(mHash_, emlen - hlen - 1)
    DB = np.array([a ^ b for a,b in zip(maskedDB, dbMask)], dtype=np.uint8)
    DB[:8*emlen - emBits] = 0
    assert DB[:emlen - hlen - slen - 2].sum() == 0
    assert DB[emlen - hlen - slen - 2] == 1
    DB = bytes(DB)
    salt = DB[-slen:]
    M_ = bytes([0x00] * 8) + mHash + salt
    assert len(M_) == 8 + hlen + slen
    mHash_2 = H(M_)
    return mHash_ == mHash_2



In [222]:
M = "Ala ma podpis"
EM = enc_emsa_pss(M, slen=256//8, emBits=2047, emlen=2048//8)
print(EM)
dec_emsa_pss(EM, M, slen=256//8, emBits=2047, emlen=2048//8)

b'\x00\x90`T)\xd8T!G\xd4\xcbl\xd0\xe9Or\xde\xb0c\x99\x1c\x82\x17g\x84\x9e\xe7.\xdfy\x03\xee\xa9\x90`T)\xd8T!G\xd4\xcbl\xd0\xe9Or\xde\xb0c\x99\x1c\x82\x17g\x84\x9e\xe7.\xdfy\x03\xee\xa9\x90`T)\xd8T!G\xd4\xcbl\xd0\xe9Or\xde\xb0c\x99\x1c\x82\x17g\x84\x9e\xe7.\xdfy\x03\xee\xa9\x90`T)\xd8T!G\xd4\xcbl\xd0\xe9Or\xde\xb0c\x99\x1c\x82\x17g\x84\x9e\xe7.\xdfy\x03\xee\xa9\x90`T)\xd8T!G\xd4\xcbl\xd0\xe9Or\xde\xb0c\x99\x1c\x82\x17g\x84\x9e\xe7.\xdfy\x03\xee\xa9\x90`T)\xd8T!G\xd4\xcbl\xd0\xe9Or\xde\xb0c\x99\x1c\x82\x17g\x84\x9e\xe7.\xdfy\x02T\x10\xb4\x98<\xbb/\xfe\x83kl\xf33,b4\xf1en\xea\xc9K\x87`IHd\xe4\xabE\x0b%]a\x94\x01\xc5\x10\x00\xcd\x91\x97\xa3\x82\xc7\x9e\x98\x1d\x01\x1c\x07\x8d\xb1\x87\xc1\x0e\xc1`5)S\x98l\t\xbc'


True

## RSA - KEM

RSA - KEM, czyli z ang. *Key Encapsulation Mechanism* to prosty algorytm dedykowany do wymiany klucza symetrycznego. Obie strony dysponują uzgodnioną funkcją skótu H. Instancja, która chce **otrzymać** tajny klucz do komunikacji symetrycznej generuje klucze RSA i udostępnia swój klucz publiczny. Instancja, która chce wygenerować tajny klucz do komunikacji symetrycznej dysponuje kluczem publicznym instancji, która chce go otrzymać.

Instancja generująca klucz symetryczny:

1) Znajdź losową liczbę $RAND$ spełniającą warunki OAEP.

2) Oblicz: $KEY = H(RAND)$. Jeżeli trzeba, przytnij $KEY$ do odpowiedniej długości.

3) Oblicz: $CIPHERED\_KEY = RSA\_OAEP\_ENCODING(KEY, (e, n))$.

4) Wyślij $CIPHERED\_KEY$.

Instancja otrzymująca zaszyfrowany klucz symetryczny:

1) Oblicz: $KEY = RSA\_OAEP\_DECODING(CIPHERED\_KEY, (d, n))$

2) Jeżeli trzeba przytnij $KEY$ do odpowiedniej długości.

Np. AES występuje w wersji 128b, 192b i 256b. Jeżeli jako H przyjmiemy więc SHA-256, nie trzeba przycinać klucza dla algorytmu AES-256. W przeciwnym razie należy klucz odpowiednio przyciąć (z lewej lub prawej, byle obie strony tak samo) i to ta wartość staje się kluczem symetrycznym.

**Zadanie 8**

Zasymuluj takową wymianę (bez przycinania klucza).

In [223]:
# Your code here

H = lambda mb: hl.sha3_256(mb).digest()
def gen_ins(pub_k):
    rand = cu.Random.get_random_bytes(32)
    key = H(rand)
    return key, len(key), enc_with_oaep(key, pub_k)

def res_ins(secret, priv_K, mlen):
    key = dec_with_oaep(secret, priv_k, mlen=mlen)
    return key

In [224]:
key, mlen, secret = gen_ins(pub_k)
key2 = res_ins(secret, priv_k, mlen)

assert key == key2